# Лабораторная работа №7

Корзун В. А. Вариант 3; 17.05.2024

# Создание текстового файла большого объёма

## Постановка задачи

Напишите пользовательскую функцию

In [ ]:
file_gen(file_name,
        file_size: int,
        type_symbols: str,
        number_words: (int,int),
        number_symbols: (int,int)
        

для создания текстового файла заданного имени и размера, строки которого будут
состоять из случайного количества слов случайной длины и составленных из случайных
символов. Слова в строке разделены одним пробелом.

Значения аргументов:

- file_name определяет имя текстового файла;
- file_size определяет размер текстового файла в мегабайтах;
- type_symbols может принимать одно из трех значений: 'digits', 'latin', 'cyrillic':
    + если type_symbols='digits', то для создания файла разрешены только цифровые символы;
    + если type_symbols='latin', то для создания файла разрешены только прописные латинские cимволы от a до z ; cтандартное значение type_symbols='latin';
    + если type_symbols='cyrillic', то для создания файла разрешены только прописные кириллические символы;
- number_words является кортежем из двух целых чисел для задания количества слов в строке; количество слов в каждой строке файла определяется как случайное число из диапазона, определенного кортежем; стандартное значение number_words=(10,50);
- number_symbols является кортежем из двух целых чисел для задания количества символов в слове; количество символов в каждом слове файла определяется как случайное число из диапазона, определенного кортежем; стандартное значение number_symbols=(5,9).

## 1.  Запись данных в текстовый файл

In [5]:
from time import process_time
from random import randint

Откроем файл с именем test.txt в текстовом режиме для записи:

In [4]:
file_name = 'test.txt'
f = open(file_name, 'w')

Создадим список, состоящий из некоторых строковых объектов, каждый из которых
заканчивается символом новой строки \n:

In [5]:
lines_list = [f'{x}\n' for x in 'test']
lines_list

['t\n', 'e\n', 's\n', 't\n']

Запишем созданные строки в файл с помощью метода writelines файлового
объекта f:

In [6]:
f.writelines(lines_list)

Вызовем метод close файлового объекта f для прекращения его связи с внешним
файлом, освобождения системных ресурсов и сброса буферизованного вывода на диск,
если он находится в памяти:

In [7]:
f.close()

Прочитаем записанные данные. Для этого откроем файл в режиме для чтения 'r':

In [8]:
with open(file_name,'r') as f:
    for line in f:
        print(line, end='')

t
e
s
t


В случае записи в файл больших объемов данных целесообразнее использовать не
списки для хранения строк, а генераторные объекты:

In [9]:
lines_gen_expr = (f'{x}\n' for x in range(10**4))
def lines_gen_fun(number_lines=10**6):
    yield from (f'{x}\n' for x in range(number_lines))
with open(file_name,'w') as f:
    f.writelines(lines_gen_expr)
    f.writelines(lines_gen_fun())

В дальнейшем будем создавать данные для записи в файл с помощью генераторной функции.

Добавим в функцию lines_gen_fun подсчет длины записанной информации в мегабайтах. Полагаем, что длина одного символа равна одному байту:

In [10]:
def lines_gen_fun(number_lines = 10**4):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
    print(f'{file_size/1024**2} Mb')

with open(file_name,'w') as f:
    f.writelines(lines_gen_fun())

0.04662513732910156 Mb


Допущение, что размер символа в строке равен одному байту, не всегда верно. В частности, для кириллических символов это не так. Более правильным решением для подсчета длины записанной информации в байтах будет использование метода encode для записываемых строк. В этом случае в коде выше вместо len(line) стоит вызывать len(line.encode('utf8')):

In [11]:
len('f'), len('ц'), len('f'.encode('utf8')), len('ц'.encode('utf8'))

(1, 1, 1, 2)

Измеряем время, необходимое для создания файла с помощью функции lines_gen_fun:

In [12]:
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun())
    end = process_time()
    print(end - start, 'секунд')

0.04662513732910156 Mb
0.015625 секунд


Внесем изменения в генераторную функцию lines_gen_fun, чтобы она отображала
процент записанных данных в файл, если аргумент status=True. Это существенно
увеличит время выполнения функции!

In [13]:
def lines_gen_fun(number_lines=10**5, status=False):
    file_size = 0
    for x in range(number_lines):
        line = f'{x}\n'
        yield line
        file_size += len(line)
        # отображение статуса записи в файл в процентах
        if status:
            status_number = x/number_lines*10**2
            print(f'\r{int(status_number)}%', end='', flush=True) # \r возврат курсора в начало
    print(f'\r100%')
    print(f'\n {file_size/1024**2} Mb') 

Запишем данные в файл:

In [14]:
# предварительно вызовите в Anaconda Prompt
# jupyter lab --NotebookApp.iopub_data_rate_limit=1.0e10
with open(file_name,'w') as f:
    start = process_time()
    f.writelines(lines_gen_fun(3*10**3, status=True))
    end = process_time()
    print(f'{end - start} секунд')

100%

 0.013246536254882812 Mb
3.0625 секунд


## 2. Генерация строк

Определим значения для переменных, которые задают параметры создания строки для
записи в файл:

In [15]:
symbols_ord = (ord('a'), ord('z')) # соответствует type_symbols = 'latin'
number_words = (10, 50)
number_symbols = (5, 9)

Создадим одно слово случайной длины из интервала $[5,9]$, состоящее из случайно
выбранных латинских символов:

In [16]:
len_word = randint(*number_symbols)
word = ''.join([chr(randint(*symbols_ord)) for i in range(len_word)])
word

'bnailvb'

$\color{Red}{Напишем~код}$ для генерации строки со случайным количеством слов, случайной
длиной слова и случайным набором символов в слове для трех множеств смволов:
прописные латинские символы, цифровые символы, прописные кириллические
символы. $\color{Red}{Протестируем}$ написанный код.

In [17]:
len_line = randint(*number_words)
line = ''
for _ in range(len_line):
    len_word = randint(*number_symbols)
    word = ''.join([chr(randint(*symbols_ord)) for i in range(len_word)])
    line += word + ' '
line = line[:-1] + '\n'
line

'dmyxl bzwvbwc gxablaib ksrzene iutifrn uizoc psrmxnfr nctwbd cmqlbkmq fneejyypu wipbqqiu uljbfkcn bkcbchnro tbyct fmpxbo xmvlkdo pwvwihe chzmgnd bepif mwhqzq ofigfvwsh mflhria vztnp bsckdwv colmqdos eythscdc wpecerkfh qgpxhnfe xsjaxdap svxmyynfh sxgfv zblzubij\n'

Теперь для числовых и кириллических:

In [18]:
symbols_ord = (ord('а'), ord('я')) # соответствует type_symbols = 'cyrillic'
len_line = randint(*number_words)
line = ''
for _ in range(len_line):
    len_word = randint(*number_symbols)
    word = ''.join([chr(randint(*symbols_ord)) for i in range(len_word)])
    line += word + ' '
line = line[:-1] + '\n'
line

'йзвпышчйь пощсфыкрз гпсээл южэхжобх жпрба нзжгф щбзпе юллаъ умюовкэ увгтщююн угпфезю фнравро ыпгмпевюж гящлыюз яхьяз кмаязциэ лшуыфшй йгнначц пэтмлхюг ьсътщ щотэхыжъ бжйрмс\n'

Работает, тогда попробуем взломать пентагон:

In [19]:
symbols_ord = (ord('0'), ord('9')) # соответствует type_symbols = 'digits'
len_line = randint(*number_words)
line = ''
for _ in range(len_line):
    len_word = randint(*number_symbols)
    word = ''.join([chr(randint(*symbols_ord)) for i in range(len_word)])
    line += word + ' '
line = line[:-1] + '\n'
line

'3544471 2301770 13307129 543774992 427030165 11978 05633 793085 082899 5246590 767480049 872151698 784708946 787347247 773873440 4942118 407148 594002782 487521835 2910521 809154 885074 08373148 2974274 143900 264209 97604 4925161 78061020 175239 03307140 55440380 33641067 74879 56672630 86811321 61856 2848608 43921067 98931936 776462 700886384\n'

Круто, тестируйте сами, у меня всё работает

<!-- Хехе, добавим экзотики: -->
<span id='block' style='color:red'>Напишем</span> генераторную функцию lines_gen_fun_v2(file_size, type_symbols, number_words, number_symbols, открой меня) для генерации строк, суммарный размер которых в мегабайтах равен file_size. <!-- А js есть?: -->
<script>
document.getElementsById('block')[0].style.color = 'red';
alert('message');
</script><!--мдааа, скуучнооо :(-->

In [3]:
def lines_gen_fun_v2(file_size, type_symbols, number_words, number_symbols):
    """Генерирует поочерёдно строки со случайным кол-вом символов в слове и со случайным кол-вом слов

    Arguments:

    file_size: размер файла в мегабайтах

    type_symbols: тип символов, digits для чисел, latin для латиницы, cyrillic для кириллицы

    number_words: кортеж обозначающий минимум и максимум кол-в слов в каждой строке

    number_symbols: кортеж обозначающий минимум и максимум кол-в символов в каждом слове

    Yield: lines
    """
    size = 0
    symbols_ord = (0,1)
    if type_symbols[0] == 'd':
        symbols_ord = (ord('0'), ord('9'))
    elif type_symbols[0] == 'c':
        symbols_ord = (ord('а'), ord('я'))
    elif type_symbols[0] == 'l':
        symbols_ord = (ord('a'), ord('z'))
    while size < file_size * 2**20:
        len_line = randint(*number_words)
        line = ''
        for _ in range(len_line):
            len_word = randint(*number_symbols)
            word = ''.join([chr(randint(*symbols_ord)) for i in range(len_word)])
            line += word + ' '
        line = line[:-1] + '\n'
        yield line
        size += len(line.encode('utf8'))

Протестируем:

In [21]:
gen = lines_gen_fun_v2(0.005, 'l', (10, 100), (5, 15))
for line in gen:
    print(len(line), end = '; ')

506; 563; 774; 309; 114; 955; 718; 975; 1109; 

In [22]:
gen = lines_gen_fun_v2(0.0005, 'c', (5, 20), (5, 6))
for line in gen:
    print(line)

олуяш рнзжб шфюэе дмшдхг жупьчв ибяымф чнгкс югихню мвлеээ ыицпхт юмыцъ мбждхй ойбщеб

хмцээ фсаэшъ эржыш ебюлч фнйспф тдшрб дункк бытуф щсншю лекгэй тъбцо щбъевл

втмяъл хюдмпю швцзнр йснтлэ мчфхй чшхуб поьчфб уемшою огбьц жтвадэ кэгяяъ мъвбъ

хгпсйв хбзвй нноом ащияаь кмпыщж ижврж вблцтд ютпоя



## 3. Запись сгенерированных строк в текстовый файл

<span style='color:red; font-size:35px'>Напишем</span> результирующую функцию:

file_gen(file_name, file_size: int, type_symbols: str, number_words: (int,int), number_symbols: (int,int))

Функция file_gen должна выводить на экран:

- процент записанных данных в процессе выполнения функции,
- фактический размер записанных данных в мегабайтах,
- время для выполнения кода функции

In [1]:
def file_gen(file_name, file_size = 0.05, type_symbols = 'l', number_words = (10, 50), number_symbols = (5, 9)):
    """Переписывает файл случайными словами со случайными символами со случайным кол-вом слов

    Arguments:

    file_name: название или путь к файлу

    file_size: размер файла в мегабайтах

    type_symbols: тип символов: digits, cyrillic, latin

    number_words: кортеж ограничивающий случайность кол-в слов в каждой строке

    number_symbols: кортеж указывающий минимум и максимум для кол-ва символов в каждом слове

    Returns: nothing
    """
    gen = lines_gen_fun_v2(file_size, type_symbols, number_words, number_symbols)
    with open(file_name, 'w') as f:
        start = process_time()
        count = 0
        for line in gen:
            count += len(line.encode('utf8'))
            f.write(line)
            status_number = count / (file_size * 2**20) * 100
            print(f'\r{int(status_number)}%', end='', flush=True)
        end = process_time()
        print(f'\r100%', end = '\n')
        print(f'{end - start} секунд')
        print(f'\n {count/1024**2} Mb')

In [24]:
file_gen('test.txt', 0.1)

100%
0.609375 секунд

 0.10018444061279297 Mb


In [25]:
file_gen('cyrillic.txt', 0.3, 'cyrillyc')

100%
1.109375 секунд

 0.3001556396484375 Mb


In [13]:
file_gen('digits.txt', 0.0005, 'diguts', (1, 15), (1, 5))

100%
0.03125 секунд

 0.0005369186401367188 Mb
